# Testing for E2 - using all spindles, training with all data
fistly define the UUID of the experiment to use

In [1]:
experimentId="5388ca14-a315-4598-97c2-d44175b24937"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [3]:
samplerate=200  #Should rethink this

In [4]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [5]:
#consider to include this in a function
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.type=='spindle']
annotations=annotations[annotations.duration>minDuration]
annotations=annotations[annotations.duration<maxDuration]
annotations=annotations.reset_index(drop=True)

In [6]:
#check with EDA results (not needed)
print(1-len(annotations)/33458)

0.005648873214178951


In [7]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [8]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [9]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [10]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,d51d037b-304a-48df-970e-58aac592ac87,0.018639
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,50ebd176-f601-4f02-bcd5-e0c07a362a35,0.020055
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,ad0e1cc1-5a53-45b1-b13e-ca760e58c787,0.021075
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,953522b6-a687-4555-b7bd-64e50323f539,0.018331
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,fdff7a52-c874-434c-b79a-4652e6521b61,0.019222
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,a99170bc-4dc5-4b27-84df-f565c6f142c5,0.019625
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,d4f86014-a983-4a56-940a-a53396539084,0.017824
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,b9ba925c-8862-41ae-85ac-87efe667dc99,0.017542
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,9a796814-2401-4dbb-811c-695c7c0888bb,0.015442
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,3be685c1-02ea-4a2f-8478-ca265ccc3196,0.016460


In [11]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                          \
                        count      mean       std       min       25%   
criteriumName                                                           
E1                       15.0  0.018463  0.001834  0.015442  0.017419   
E2                       15.0  0.058320  0.004520  0.052347  0.054965   
union                    15.0  0.059557  0.004579  0.053738  0.055972   

                                             
                    50%       75%       max  
criteriumName                                
E1             0.018331  0.019424  0.022439  
E2             0.056746  0.061623  0.067005  
union          0.058222  0.062711  0.068526

In [12]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [13]:
hyperClose=0.1
hyperDuration=minDuration
hyperThres=0.3
hyperDepth=40

## Testing with E2 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [14]:
experimentModels=experimentModels[experimentModels.criteriumName=='E2'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,e1d4b905-b790-4fe9-8a43-59b76a36b9dd,0.056746
1,0002,E2,[0002],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,c1f62253-8dcf-4cc8-8b95-bde893a5a09e,0.061232
2,0002,E2,[0002],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,4f00824a-e500-4ad5-8aec-b8f5322c821a,0.063062
3,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,b729b67d-261b-41e9-8555-a5749e35811e,0.061716
4,0002,E2,[0002],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,d084f1f0-06ef-468e-b82e-16b5f96fdc69,0.058705
5,0002,E2,[0002],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,e4a48e65-6ade-4c80-a360-abf73bd39eeb,0.061530
6,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,1eca4001-1d4f-4d59-b1de-1b02a0d6f95f,0.055908
7,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,c35d7164-a086-4277-899c-731c13b90a75,0.052347
8,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,d610a0a1-aae2-46e6-b31e-a91cb6c987fe,0.054834
9,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,58ce72d9-8af2-4491-8cb9-99536ad462a1,0.053001


In [15]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [16]:
dumpPickle('experimentModelsTest_E2_temp.pkl',experimentModels)

In [17]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [18]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [19]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [20]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [21]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [22]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.686322 NaN  0.686322  0.686322  0.686322            1.0   
0002     1.0  0.843391 NaN  0.843391  0.843391  0.843391            1.0   
0003     1.0  0.740260 NaN  0.740260  0.740260  0.740260            1.0   
0005     1.0  0.682175 NaN  0.682175  0.682175  0.682175            1.0   
0006     1.0  0.642082 NaN  0.642082  0.642082  0.642082            1.0   
0007     1.0  0.670033 NaN  0.670033  0.670033  0.670033            1.0   
0009     1.0  0.729994 NaN  0.729994  0.729994  0.729994            1.0   
0010     1.0  0.778363 NaN  0.778363  0.778363  0.778363            1.0   
0011     1.0  0.764591 NaN  0.764591  0.764591  0.764591            1.0   
0012     1.0  0.712846 NaN  0.712846  0.712846  0.712846            1.0   
0013     1.0  0.725914 NaN  0.725914  0.725914  0.725914            1.0   
0014     1.0  0.663061 NaN  0.663061  0.663061  0.663061            1.0   
0017     1.0  0.745741 NaN  0.745741  0.745741  0.745741            1.0   
0018     1.0  0.763137 NaN  0.763137  0.763137  0.763137            1.0   
0019     1.0  0.728929 NaN  0.728929  0.728929  0.728929            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.847037 NaN  0.847037  0.847037  0.847037         1.0  0.577065 NaN   
0002  0.790985 NaN  0.790985  0.790985  0.790985         1.0  0.903138 NaN   
0003  0.764286 NaN  0.764286  0.764286  0.764286         1.0  0.717647 NaN   
0005  0.542042 NaN  0.542042  0.542042  0.542042         1.0  0.918850 NaN   
0006  0.792857 NaN  0.792857  0.792857  0.792857         1.0  0.539490 NaN   
0007  0.557101 NaN  0.557101  0.557101  0.557101         1.0  0.838791 NaN   
0009  0.759005 NaN  0.759005  0.759005  0.759005         1.0  0.703097 NaN   
0010  0.751701 NaN  0.751701  0.751701  0.751701         1.0  0.806854 NaN   
0011  0.662300 NaN  0.662300  0.662300  0.662300         1.0  0.903289 NaN   
0012  0.618899 NaN  0.618899  0.618899  0.618899         1.0  0.839966 NaN   
0013  0.598994 NaN  0.598994  0.598994  0.598994         1.0  0.920702 NaN   
0014  0.518532 NaN  0.518532  0.518532  0.518532         1.0  0.918329 NaN   
0017  0.753661 NaN  0.753661  0.753661  0.753661         1.0  0.737995 NaN   
0018  0.671889 NaN  0.671889  0.671889  0.671889         1.0  0.882779 NaN   
0019  0.604798 NaN  0.604798  0.604798  0.604798         1.0  0.916190 NaN   

                                    
           min       50%       max  
test                                
0001  0.577065  0.577065  0.577065  
0002  0.903138  0.903138  0.903138  
0003  0.717647  0.717647  0.717647  
0005  0.918850  0.918850  0.918850  
0006  0.539490  0.539490  0.539490  
0007  0.838791  0.838791  0.838791  
0009  0.703097  0.703097  0.703097  
0010  0.806854  0.806854  0.806854  
0011  0.903289  0.903289  0.903289  
0012  0.839966  0.839966  0.839966  
0013  0.920702  0.920702  0.920702  
0014  0.918329  0.918329  0.918329  
0017  0.737995  0.737995  0.737995  
0018  0.882779  0.882779  0.882779  
0019  0.916190  0.916190  0.916190

In [23]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.686322,0.847037,0.577065
1,0002,0.843391,0.790985,0.903138
2,0003,0.740260,0.764286,0.717647
3,0005,0.682175,0.542042,0.918850
4,0006,0.642082,0.792857,0.539490
5,0007,0.670033,0.557101,0.838791
6,0009,0.729994,0.759005,0.703097
7,0010,0.778363,0.751701,0.806854
8,0011,0.764591,0.662300,0.903289
9,0012,0.712846,0.618899,0.839966


In [24]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.251227e-01
eventPrecision    6.822724e-01
eventRecall       8.082788e-01
dtype: float64

In [25]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_4160\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.051962
eventPrecision    0.104652
eventRecall       0.126508
dtype: float64

In [26]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [27]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

## Restrict to only N2 stage

In [28]:
experimentModels_N2=experimentModels.copy()
experimentModels_N2.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate,rawF1,rawPrecision,rawRecall,f1,precision,recall,eventF1,eventPrecision,eventRecall
0,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,e1d4b905-b790-4fe9-8a43-59b76a36b9dd,0.056746,0.643092,0.831351,0.524352,0.646297,0.832101,0.528324,0.686322,0.847037,0.577065
1,0002,E2,[0002],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,c1f62253-8dcf-4cc8-8b95-bde893a5a09e,0.061232,0.778292,0.744297,0.815541,0.780111,0.737629,0.827784,0.843391,0.790985,0.903138
2,0002,E2,[0002],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,4f00824a-e500-4ad5-8aec-b8f5322c821a,0.063062,0.675548,0.698052,0.654449,0.679391,0.699566,0.660347,0.740260,0.764286,0.717647
3,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,b729b67d-261b-41e9-8555-a5749e35811e,0.061716,0.622045,0.477846,0.890887,0.619382,0.473051,0.896790,0.682175,0.542042,0.918850
4,0002,E2,[0002],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,d084f1f0-06ef-468e-b82e-16b5f96fdc69,0.058705,0.591603,0.740139,0.492720,0.590872,0.740030,0.491755,0.642082,0.792857,0.539490


In [29]:
annotationsN2=pd.read_csv(datapath+'/MASS/annotations/spindlesFilteredN2.csv')
annotationsN2['samplerate']=samplerate
annotationsN2['subjectId']=annotationsN2.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
annotationsN2['labelerId']=annotationsN2.apply(
    lambda row: str(row.labelerId).zfill(4),axis=1)
annotationsN2['stopTime']=annotationsN2.apply(
    lambda row: row.startTime+row.duration , axis=1)
annotationsN2['startInd']=annotationsN2.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
annotationsN2['stopInd']=annotationsN2.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

In [30]:
## load hypnograms
stagesAnnotations=pd.read_csv(datapath+"\MASS\stages\stages.csv")
stagesAnnotations['samplerate']=samplerate
stagesAnnotations['subjectId']=stagesAnnotations.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
stagesAnnotations['stopTime']=stagesAnnotations.apply(
    lambda row: row.startTime+row.duration , axis=1)
stagesAnnotations['startInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
stagesAnnotations['stopInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)
stagesAnnotations.head(5)

hypnograms={}
for ind, row in signalsMetadata.iterrows():
    subjectId=row.subjectId
    thisStages=stagesAnnotations[stagesAnnotations.subjectId==subjectId]
    excerptDimension=int(row.duration*row.samplerate)
    thisHypnogram=np.ones((excerptDimension,))*np.nan
    for ind_stg, row_stg in thisStages.iterrows():
        thisHypnogram[row_stg.startInd:row_stg.stopInd]=row_stg.value
    hypnograms[subjectId]=thisHypnogram   

In [31]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels_N2.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels_N2)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    testHypnogram=hypnograms[testSubjectId]
    testN2Mask=testHypnogram==2
    #Define annotations criterium
    usedAnnotations=annotationsN2[annotationsN2.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres

    #Apply mask ---------------------------------->
    rawLabels=rawLabels*testN2Mask
    #<---------------------------------------------

    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)

    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels_N2['rawF1']=rawF1s
experimentModels_N2['rawPrecision']=rawPrecisions
experimentModels_N2['rawRecall']=rawRecalls

experimentModels_N2['f1']=f1s
experimentModels_N2['precision']=precisions
experimentModels_N2['recall']=recalls

experimentModels_N2['eventF1']=eventF1s
experimentModels_N2['eventPrecision']=eventPrecisions
experimentModels_N2['eventRecall']=eventRecalls


*************************
1 of 15
*************************
2 of 15
*************************
3 of 15
*************************
4 of 15
*************************
5 of 15
*************************
6 of 15
*************************
7 of 15
*************************
8 of 15
*************************
9 of 15
*************************
10 of 15
*************************
11 of 15
*************************
12 of 15
*************************
13 of 15
*************************
14 of 15
*************************
15 of 15


In [32]:
dumpPickle('experimentModelsTest_N2_E2_temp.pkl',experimentModels_N2)

In [33]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [34]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [35]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [36]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [37]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.716014 NaN  0.716014  0.716014  0.716014            1.0   
0002     1.0  0.866606 NaN  0.866606  0.866606  0.866606            1.0   
0003     1.0  0.788018 NaN  0.788018  0.788018  0.788018            1.0   
0005     1.0  0.747412 NaN  0.747412  0.747412  0.747412            1.0   
0006     1.0  0.673252 NaN  0.673252  0.673252  0.673252            1.0   
0007     1.0  0.784012 NaN  0.784012  0.784012  0.784012            1.0   
0009     1.0  0.738625 NaN  0.738625  0.738625  0.738625            1.0   
0010     1.0  0.861827 NaN  0.861827  0.861827  0.861827            1.0   
0011     1.0  0.800000 NaN  0.800000  0.800000  0.800000            1.0   
0012     1.0  0.824255 NaN  0.824255  0.824255  0.824255            1.0   
0013     1.0  0.774992 NaN  0.774992  0.774992  0.774992            1.0   
0014     1.0  0.713333 NaN  0.713333  0.713333  0.713333            1.0   
0017     1.0  0.798624 NaN  0.798624  0.798624  0.798624            1.0   
0018     1.0  0.796327 NaN  0.796327  0.796327  0.796327            1.0   
0019     1.0  0.723936 NaN  0.723936  0.723936  0.723936            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.943614 NaN  0.943614  0.943614  0.943614         1.0  0.577114 NaN   
0002  0.833407 NaN  0.833407  0.833407  0.833407         1.0  0.902520 NaN   
0003  0.869919 NaN  0.869919  0.869919  0.869919         1.0  0.720067 NaN   
0005  0.629220 NaN  0.629220  0.629220  0.629220         1.0  0.919492 NaN   
0006  0.893145 NaN  0.893145  0.893145  0.893145         1.0  0.540244 NaN   
0007  0.737322 NaN  0.737322  0.737322  0.737322         1.0  0.836538 NaN   
0009  0.779442 NaN  0.779442  0.779442  0.779442         1.0  0.701840 NaN   
0010  0.926888 NaN  0.926888  0.926888  0.926888         1.0  0.805541 NaN   
0011  0.716795 NaN  0.716795  0.716795  0.716795         1.0  0.904336 NaN   
0012  0.808528 NaN  0.808528  0.808528  0.808528         1.0  0.840555 NaN   
0013  0.669882 NaN  0.669882  0.669882  0.669882         1.0  0.918681 NaN   
0014  0.583502 NaN  0.583502  0.583502  0.583502         1.0  0.916931 NaN   
0017  0.868263 NaN  0.868263  0.868263  0.868263         1.0  0.739389 NaN   
0018  0.725137 NaN  0.725137  0.725137  0.725137         1.0  0.882817 NaN   
0019  0.598706 NaN  0.598706  0.598706  0.598706         1.0  0.914370 NaN   

                                    
           min       50%       max  
test                                
0001  0.577114  0.577114  0.577114  
0002  0.902520  0.902520  0.902520  
0003  0.720067  0.720067  0.720067  
0005  0.919492  0.919492  0.919492  
0006  0.540244  0.540244  0.540244  
0007  0.836538  0.836538  0.836538  
0009  0.701840  0.701840  0.701840  
0010  0.805541  0.805541  0.805541  
0011  0.904336  0.904336  0.904336  
0012  0.840555  0.840555  0.840555  
0013  0.918681  0.918681  0.918681  
0014  0.916931  0.916931  0.916931  
0017  0.739389  0.739389  0.739389  
0018  0.882817  0.882817  0.882817  
0019  0.914370  0.914370  0.914370

In [38]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.716014,0.943614,0.577114
1,0002,0.866606,0.833407,0.902520
2,0003,0.788018,0.869919,0.720067
3,0005,0.747412,0.629220,0.919492
4,0006,0.673252,0.893145,0.540244
5,0007,0.784012,0.737322,0.836538
6,0009,0.738625,0.779442,0.701840
7,0010,0.861827,0.926888,0.805541
8,0011,0.800000,0.716795,0.904336
9,0012,0.824255,0.808528,0.840555


In [39]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.738157e-01
eventPrecision    7.722513e-01
eventRecall       8.080291e-01
dtype: float64

In [40]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_4160\813125388.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.055130
eventPrecision    0.118006
eventRecall       0.126000
dtype: float64

In [41]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels_N2.eventPrecision,
    'event F1':experimentModels_N2.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels_N2.eventRecall,
    'event F1':experimentModels_N2.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [42]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### remove outliers

In [43]:
np.setdiff1d(experimentModels_N2.test,['0006'])

array(['0001', '0002', '0003', '0005', '0007', '0009', '0010', '0011',
       '0012', '0013', '0014', '0017', '0018', '0019'], dtype=object)

In [44]:
experimentModels_N2[experimentModels_N2.test.isin(np.setdiff1d(experimentModels_N2.test,['0001','0014','0019']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              1.666917e+43
eventF1           7.878293e-01
eventPrecision    7.881624e-01
eventRecall       8.093351e-01
dtype: float64

In [45]:
annotationPairToGraph(gtAnnotations,processedAnnotations)

NameError: name 'px' is not defined